## Installing Modules

In [1]:
!pip install google_trans_new
!pip install spacy==3
!python -m spacy download en_core_web_sm
!pip install pytreebank

2021-06-03 16:19:30.528126: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Imports

In [2]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import pandas as pd
import sys, os, pickle
import numpy as np

import pytreebank

import spacy
nlp = spacy.load('en_core_web_sm')

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

## Loading Data

In [3]:
# !gdown --id 1HmYahgrwNcZREWtUTr6H11ygJufuFcTc
# !gdown --id 14hb3DlvmMeEvWhNYXAZjhE3MFS1T8Nte

# !gdown --id 1wIIPVlZbHa9-YBSEDRb_P_2ChZ8a12Y-
# !gdown --id 1TXcS-WBy0yEYx7YkS3SjPkRTylSXamCq

In [4]:
# labels = []
# sentences = []
# with open('/content/train.txt') as f:
#     for line in f:
#         div = line.index(' ')
#         sentences.append((line[div+1:]))
#         labels.append(line[:div])

# df = pd.DataFrame(list(zip(sentences, labels)),
#                columns =['sentence', 'label'])


In [5]:
out_path = os.path.join(sys.path[0], '{}.txt')
dataset = pytreebank.load_sst('./ptb')

# Store train, dev and test in separate files
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))

In [6]:
df1 = pd.read_csv('train.txt', sep='\t', header=None, names=['label', 'sentence'])
df1['label'] = df1['label']
df1['label'] = df1['label'].astype(int).astype('category')
df2 = pd.read_csv('test.txt', sep='\t', header=None, names=['label', 'sentence'])
df2['label'] = df2['label']
df2['label'] = df2['label'].astype(int).astype('category')
df3 = pd.read_csv('dev.txt', sep='\t', header=None, names=['label', 'sentence'])
df3['label'] = df3['label']
df3['label'] = df3['label'].astype(int).astype('category')
df = pd.concat([df1, df2, df3], ignore_index=True)
len(df)

11855

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [7]:
df.head()

,label,sentence
0,4,The Rock is destined to be the 21st Century 's...
1,5,The gorgeously elaborate continuation of `` Th...
2,4,Singer/composer Bryan Adams contributes a slew...
3,3,You 'd think by now America would have had eno...
4,4,Yet the act is still charming here .


In [8]:
df.shape

(11855, 2)

In [9]:
df.label.value_counts()

2    3140
4    3111
3    2242
5    1852
1    1510
Name: label, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [10]:
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [11]:
fields = [('sentence', Sentence),('label',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [12]:
example = [data.Example.fromlist([df.sentence[i],df.label[i]], fields) for i in range(df.shape[0])] 

In [13]:
stanfordDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [14]:
(train, valid) = stanfordDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [15]:
(len(train), len(valid))

(10077, 1778)

An example from the dataset:

In [16]:
vars(train.examples[10])

{'label': 2, 'sentence': ['A', 'bit', 'of', 'an', 'unwieldy', 'mess', '.']}

In [17]:
" ".join((vars(train.examples[10]))['sentence'])

'A bit of an unwieldy mess .'

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [18]:
Sentence.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [19]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18638
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 9493), (',', 8482), ('the', 7193), ('and', 5298), ('a', 5190), ('of', 5187), ('to', 3620), ('-', 3345), ("'s", 3017), ('is', 3009)]
Labels :  defaultdict(None, {2: 0, 4: 1, 3: 2, 5: 3, 1: 4})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [22]:
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [23]:
class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [24]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 100
num_hidden_nodes = 20
num_output_nodes = 5
num_layers = 4
dropout = 0.4

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [25]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(18638, 100)
  (encoder): LSTM(100, 20, num_layers=4, batch_first=True, dropout=0.4)
  (fc): Linear(in_features=20, out_features=5, bias=True)
)
The model has 1,883,745 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [26]:
# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
criterion = nn.CrossEntropyLoss()

# define metric
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [27]:
def mytrain(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sent, sent_lengths = batch.sentence   
        # sent= batch.sentence   
        
        # convert to 1D tensor
        predictions = model(sent, sent_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = categorical_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [28]:
def myevaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sent, sent_lengths = batch.sentence
            # sent= batch.sentence 
            
            # convert to 1d tensor
            predictions = model(sent, sent_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [29]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = mytrain(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = myevaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print("Running Epoch: ", epoch+1)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Running Epoch:  1
	Train Loss: 1.583 | Train Acc: 26.37%
	 Val. Loss: 1.568 |  Val. Acc: 29.65% 

Running Epoch:  2
	Train Loss: 1.563 | Train Acc: 31.82%
	 Val. Loss: 1.554 |  Val. Acc: 32.32% 

Running Epoch:  3
	Train Loss: 1.518 | Train Acc: 37.83%
	 Val. Loss: 1.542 |  Val. Acc: 34.01% 

Running Epoch:  4
	Train Loss: 1.453 | Train Acc: 44.44%
	 Val. Loss: 1.535 |  Val. Acc: 34.69% 

Running Epoch:  5
	Train Loss: 1.396 | Train Acc: 51.24%
	 Val. Loss: 1.530 |  Val. Acc: 35.42% 

Running Epoch:  6
	Train Loss: 1.338 | Train Acc: 58.02%
	 Val. Loss: 1.527 |  Val. Acc: 35.54% 

Running Epoch:  7
	Train Loss: 1.280 | Train Acc: 64.18%
	 Val. Loss: 1.536 |  Val. Acc: 34.59% 

Running Epoch:  8
	Train Loss: 1.230 | Train Acc: 69.00%
	 Val. Loss: 1.531 |  Val. Acc: 35.75% 

Running Epoch:  9
	Train Loss: 1.192 | Train Acc: 72.66%
	 Val. Loss: 1.547 |  Val. Acc: 33.70% 

Running Epoch:  10
	Train Loss: 1.160 | Train Acc: 75.72%
	 Val. Loss: 1.539 |  Val. Acc: 34.57% 



## Model Testing

In [30]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 



def classify_tweet(tweet):
    
    categories = {0: "Worst", 1:"Negative", 2:"Neutral", 3:"Positive", 4:"Great"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    # return categories[pred.item()]
    return pred.item()

In [31]:
classify_tweet("This is something you will regret.")

2

In [32]:
for i in np.random.randint(0,len(valid),10):
  sent = " ".join((vars(valid.examples[i]))['sentence'])
  pred = classify_tweet(sent)
  label = (vars(valid.examples[i]))['label']
  print(f'Sentence: {sent[:60]} \t Predicted: {pred} \t Actual: {label}')
  

Sentence: The bottom line , at least in my opinion , is Imposter makes 	 Predicted: 2 	 Actual: 2
Sentence: It may be a somewhat backhanded compliment to say that the f 	 Predicted: 2 	 Actual: 4
Sentence: You want to call Domino 's . 	 Predicted: 0 	 Actual: 3
Sentence: ... there is enough originality in ` Life ' to distance it f 	 Predicted: 1 	 Actual: 4
Sentence: Easier to respect than enthuse over , Andersson 's rigorous  	 Predicted: 0 	 Actual: 3
Sentence: Just one bad idea after another . 	 Predicted: 0 	 Actual: 2
Sentence: Murder by Numbers just does n't add up . 	 Predicted: 0 	 Actual: 2
Sentence: We learn a lot about dying coral and see a lot of life on th 	 Predicted: 1 	 Actual: 3
Sentence: An even more predictable , cliche - ridden endeavor than its 	 Predicted: 2 	 Actual: 2
Sentence: Just one bad idea after another . 	 Predicted: 0 	 Actual: 2


## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [33]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [34]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [35]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [36]:
!pip install googletrans==3.1.0a0

In [37]:
import googletrans
from googletrans import Translator

In [38]:
from google_trans_new import google_translator  
translator = google_translator()  

In [39]:
# translator = googletrans.Translator()
sentence = ['I am a good man']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, lang_tgt=trans_lang) 
print(translations)

en_text = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
print(en_text)

Translating to armenian
[«Ես լավ մարդ եմ»] 
["I'm a good person"] 


In [40]:
def mytranslator(sentence, trans_lang):
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  en_text = translator.translate(translations, lang_src=trans_lang, lang_tgt='en')  
  return en_text


In [41]:
en_text = mytranslator(sentence, trans_lang)
print(en_text)

["I'm a good person"] 


## Augmented Dataset Generation

In [42]:
from sklearn.utils import shuffle

def augmenter(df,samples=300):
    new_sentence=[]
    new_label=[]
    

    ## data augmentation loop
    for i in np.random.randint(0,len(df),samples):
        
            sentence = df.iloc[i]['sentence']
            augmented_sentence = mytranslator(sentence, trans_lang)
            new_sentence.append(augmented_sentence)
            label = df.iloc[i]['label']
            new_label.append(label)
    
    
    ## dataframe
    new=pd.DataFrame({'sentence':new_sentence,'label':new_label})
    dfn=shuffle(df.append(new).reset_index(drop=True))
    return dfn

In [43]:
dfn = augmenter(df,200)

In [44]:
dfn.head()

,label,sentence
385,4,"Strange it is , but delightfully so ."
3137,4,Splendidly illustrates the ability of the huma...
10121,5,Excellent acting and direction .
1798,3,"With wit and empathy to spare , waydowntown ac..."
3785,3,But something seems to be missing .


In [45]:
dfn.shape

(12055, 2)

In [46]:
example = [data.Example.fromlist([dfn.sentence[i],dfn.label[i]], fields) for i in range(dfn.shape[0])] 
stanfordDataset = data.Dataset(example, fields)
(train, valid) = stanfordDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))
(len(train), len(valid))

(10247, 1808)

In [47]:
vars(train.examples[10])

{'label': 2, 'sentence': ['A', 'bit', 'of', 'an', 'unwieldy', 'mess', '.']}

In [48]:
Sentence.build_vocab(train)
Label.build_vocab(train)

In [49]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  18700
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 9665), (',', 8570), ('the', 7346), ('and', 5333), ('of', 5280), ('a', 5211), ('to', 3659), ('-', 3395), ('is', 3057), ("'s", 3037)]
Labels :  defaultdict(None, {2: 0, 4: 1, 3: 2, 5: 3, 1: 4})


In [50]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [51]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 8
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [52]:
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [53]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = mytrain(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = myevaluate(model, valid_iterator, criterion)
    # valid_loss, valid_acc = train(model, valid_iterator, optimizer, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print("Running Epoch: ", epoch+1)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

Running Epoch:  1
	Train Loss: 1.610 | Train Acc: 20.74%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  2
	Train Loss: 1.610 | Train Acc: 20.81%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  3
	Train Loss: 1.610 | Train Acc: 20.77%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  4
	Train Loss: 1.610 | Train Acc: 20.88%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  5
	Train Loss: 1.610 | Train Acc: 20.84%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  6
	Train Loss: 1.610 | Train Acc: 20.77%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  7
	Train Loss: 1.610 | Train Acc: 20.74%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  8
	Train Loss: 1.610 | Train Acc: 20.74%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  9
	Train Loss: 1.610 | Train Acc: 20.77%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

Running Epoch:  10
	Train Loss: 1.610 | Train Acc: 20.77%
	 Val. Loss: 1.612 |  Val. Acc: 20.07% 

